# 📧 Email Assistant Chatbot Prototype

## Project Objectives
- Create an intelligent email drafting assistant
- Utilize AI for context-aware email generation
- Implement research-backed email composition

## Required APIs
- Gemini AI for draft generation
- Tavily for contextual research
- Postgress for database connection

In [ ]:
%%capture --no-stderr

In [ ]:
%pip install -U langchain langgraph langchain_google_genai langchain_community tavily_python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.7/135.7 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.2 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.25
    Uninstalling langchain-core-0.3.25:
      Successfully uninstalled langchain-core-0.3.25
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.12
    Uninstalling langchain-0.3.12:
      Successfully uninstalled langchain-0.3.12


In [ ]:
# Install required libraries
!pip install -q google-generativeai tavily-python

In [ ]:
!pip install -q \
    fastapi \
    uvicorn \
    sqlalchemy \
    pydantic \
    psycopg2-binary \
    python-dotenv \
    google-generativeai \
    tavily-python \
    python-jose \
    passlib \
    "uvicorn[standard]" \
    nest_asyncio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 48.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.6/525.6 kB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.3/149.3 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.1/442.1 kB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 92.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 443.8/443.8 kB 36.8 MB/s eta 0:00:00


In [ ]:
!pip install pyngrok

In [ ]:
# Importing essential libraries
import nest_asyncio
nest_asyncio.apply()

import uvicorn
import asyncio
import google.generativeai as genai
from tavily import TavilyClient
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from fastapi.middleware.cors import CORSMiddleware
from typing import List, Dict, Optional
import sqlalchemy
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine, Column, Integer, String, Text, DateTime
from sqlalchemy import inspect
from datetime import datetime
from google.colab import userdata
import os
from typing import List, Dict, Any
import google.generativeai as genai
import tavily
from sqlalchemy.sql import func
from pyngrok import ngrok


In [ ]:
app = FastAPI()

# Add CORS middleware
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # In production, replace with your Next.js domain
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

In [ ]:
# Securely retrieve API keys
GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
TAVILY_API_KEY = userdata.get('TAVILY_API_KEY')
DATABASE_URL = userdata.get('DATABASE_URL')
NGROK_AUTH_TOKEN = userdata.get('NGROK_AUTH_TOKEN')

# Validate API keys
import os
os.environ['GEMINI_API_KEY'] = GEMINI_API_KEY
os.environ['TAVILY_API_KEY'] = TAVILY_API_KEY
os.environ['DATABASE_URL'] = DATABASE_URL
os.environ['NGROK_AUTH_TOKEN'] = NGROK_AUTH_TOKEN

print("API Keys configured successfully!")

API Keys configured successfully!


In [ ]:
# Database Configuration
Base = declarative_base()
engine = create_engine(os.environ.get('DATABASE_URL'))
SessionLocal = sessionmaker(bind=engine)

# Database Model
class EmailDraft(Base):
    """SQLAlchemy model for email drafts"""
    __tablename__ = 'email_drafts'

    id = Column(Integer, primary_key=True, index=True)
    content = Column(Text, nullable=False)
    recipient = Column(String(255), nullable=False)
    context = Column(Text)
    timestamp = Column(DateTime(timezone=True), server_default=func.now())
    status = Column(String(50), default='draft')
    draft_version = Column(Integer, default=1)

    def to_dict(self):
        """Convert model instance to dictionary"""
        return {
            'id': self.id,
            'content': self.content,
            'recipient': self.recipient,
            'context': self.context,
            'timestamp': self.timestamp,
            'status': self.status,
            'draft_version': self.draft_version
        }

class EmailAssistantChatbot:
    def __init__(self):
        # Create tables if not exists
        Base.metadata.create_all(bind=engine)

        # Gemini AI Configuration
        gemini_api_key = os.getenv('GEMINI_API_KEY')
        if not gemini_api_key:
            raise ValueError("GEMINI_API_KEY environment variable not set")
        genai.configure(api_key=gemini_api_key)
        self.model = genai.GenerativeModel('gemini-pro')

        # Tavily Search Configuration
        tavily_api_key = os.getenv('TAVILY_API_KEY')
        if not tavily_api_key:
            raise ValueError("TAVILY_API_KEY environment variable not set")
        self.tavily_client = TavilyClient(api_key=tavily_api_key)

    def generate_email_draft(self, context: str) -> str:
        """Generate email draft using Gemini AI"""
        prompt = f"""Help draft a professional email based on the following context:
        {context}

        Guidelines:
        - Maintain a clear, professional tone
        - Address the key points directly
        - Provide a structured email draft
        - Include appropriate salutation and closing"""

        try:
            response = self.model.generate_content(prompt)
            return response.text
        except Exception as e:
            return f"Error generating email draft: {str(e)}"

    def research_email_context(self, context: str) -> List[Dict[str, Any]]:
        """Use Tavily for contextual research"""
        try:
            search_results = self.tavily_client.search(query=context)
            return search_results.get('results', [])
        except Exception as e:
            print(f"Search error: {str(e)}")
            return []

    def save_email_draft(self, content: str, recipient: str, context: str) -> int:
        """Save email draft to database"""
        session = SessionLocal()
        try:
            new_draft = EmailDraft(
                content=content,
                recipient=recipient,
                context=context
            )
            session.add(new_draft)
            session.commit()
            session.refresh(new_draft)
            return new_draft.id
        except Exception as e:
            session.rollback()
            print(f"Error saving draft: {str(e)}")
            return -1
        finally:
            session.close()

    def get_email_draft(self, draft_id: int) -> Dict[str, Any]:
        """Retrieve an email draft by ID"""
        session = SessionLocal()
        try:
            draft = session.query(EmailDraft).filter(EmailDraft.id == draft_id).first()
            return draft.to_dict() if draft else {}
        except Exception as e:
            print(f"Error retrieving draft: {str(e)}")
            return {}
        finally:
            session.close()

    def analyze_email_context(self, context: str) -> Dict[str, Any]:
        """Comprehensive email context analysis"""
        research_results = self.research_email_context(context)
        draft = self.generate_email_draft(context)
        draft_id = self.save_email_draft(draft, 'recipient@example.com', context)

        return {
            'draft_id': draft_id,
            'draft_content': draft,
            'research_insights': research_results
        }

<ipython-input-9-855436c65cc3>:2: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [ ]:
# Initialize the Email Assistant
assistant = EmailAssistantChatbot()

# Define email drafting scenarios
email_contexts = [
    "Write a follow-up email after a job interview",
    "Send a project update to team members",
    "Request an extension for a deadline"
]

# Demonstrate email drafting for different contexts
for context in email_contexts:
    print(f"\n--- Email Draft for Context: {context} ---")

    # Analyze context and generate draft
    result = assistant.analyze_email_context(context)

    print("Generated Draft:")
    print(result['draft_content'])

    print("\nResearch Insights:")
    for insight in result['research_insights']:
        print(f"- {insight['title']}: {insight['url']}")

@app.post("/example-drafts")
async def get_example_drafts(context):
    """
    Generate example email drafts using predefined contexts
    """
    if not context:
        raise HTTPException(status_code=400, detail="Context is required")

    try:
        email_contexts = [
            context
        ]

        results = []
        for context in email_contexts:
            result = assistant.analyze_email_context(context)
            results.append({
                "context": context,
                "analysis": result
            })

        return {"examples": results}
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

# Error handling middleware
@app.middleware("http")
async def error_handling_middleware(request, call_next):
    try:
        return await call_next(request)
    except Exception as e:
        return HTTPException(
            status_code=500,
            detail=f"Internal server error: {str(e)}"
        )

# Health check endpoint
@app.get("/health")
async def health_check():
    return {"status": "healthy", "service": "email-assistant"}

# class EmailContext(BaseModel):
#     context: str

# class ResearchInsight(BaseModel):
#     title: str
#     url: str
#     relevance_score: float

# class EmailAnalysis(BaseModel):
#     draft_content: str
#     research_insights: List[ResearchInsight]

# class EmailAssistantChatbot:
#     def analyze_email_context(self, context: str) -> dict:
#         # Simulate email generation based on context
#         drafts = {
#             "job interview": """
# Dear [Interviewer's Name],

# Thank you for taking the time to meet with me yesterday regarding the [Position] role at [Company]. I enjoyed learning more about the position and the team's goals.

# I wanted to follow up on our discussion about [specific topic discussed]. Your insights about [company initiative] were particularly interesting, and I believe my experience with [relevant skill] would be valuable in helping achieve those objectives.

# Please let me know if you need any additional information from me. I'm very excited about the opportunity to join the team.

# Best regards,
# [Your Name]""",

#             "project update": """
# Hi team,

# I wanted to provide a quick update on the [Project Name] status:

# Key Accomplishments:
# - Completed [milestone] ahead of schedule
# - Resolved [issue] that was blocking progress
# - Implemented new [feature/process]

# Next Steps:
# We're focusing on [upcoming milestone] and expect to complete it by [date].

# Please let me know if you have any questions or concerns.

# Best,
# [Your Name]""",

#             "deadline extension": """
# Dear [Name],

# I'm writing regarding the upcoming deadline for [project/assignment] due on [original date].

# Due to [specific reason], I would like to request a [timeframe] extension. This additional time will ensure I can deliver high-quality work that meets all requirements.

# I have already completed [progress details] and can submit a preliminary version by the original deadline if needed.

# Thank you for considering this request.

# Best regards,
# [Your Name]"""
#         }

#         # Determine the most relevant template
#         template_key = "job interview"  # Default
#         for key in drafts.keys():
#             if key in context.lower():
#                 template_key = key
#                 break

#         # Generate research insights
#         insights = [
#             ResearchInsight(
#                 title="Best Practices for Professional Emails",
#                 url="https://example.com/email-best-practices",
#                 relevance_score=0.95
#             ),
#             ResearchInsight(
#                 title="Email Templates and Guidelines",
#                 url="https://example.com/email-templates",
#                 relevance_score=0.85
#             )
#         ]

#         return {
#             "draft_content": drafts[template_key],
#             "research_insights": insights
#         }

# @app.post("/example-drafts")
# async def get_example_drafts(context: EmailContext):
#     """
#     Generate example email drafts using predefined contexts
#     """
#     try:
#         assistant = EmailAssistantChatbot()
#         result = assistant.analyze_email_context(context.context)
#         return {
#             "examples": [{
#                 "context": context.context,
#                 "analysis": result
#             }]
#         }
#     except Exception as e:
#         raise HTTPException(status_code=500, detail=str(e))

# @app.middleware("http")
# async def error_handling_middleware(request, call_next):
#     try:
#         return await call_next(request)
#     except Exception as e:
#         return HTTPException(
#             status_code=500,
#             detail=f"Internal server error: {str(e)}"
#         )

# @app.get("/health")
# async def health_check():
#     return {"status": "healthy", "service": "email-assistant"}


--- Email Draft for Context: Write a follow-up email after a job interview ---
Generated Draft:
**Subject: Follow-Up to Interview for [Position Name]**

Dear [Hiring Manager Name],

I hope this email finds you well.

I am writing to follow up on my interview for the [position name] position with [company name] on [date]. I greatly appreciated the opportunity to meet with you and learn more about the role and the company.

During the interview, I was particularly impressed by [mention specific aspect(s) discussed]. I am confident that my skills and experience in [list relevant skills] would make me a valuable addition to your team.

I am eager to discuss my qualifications further and explore how I can contribute to the success of [company name]. I am available for any additional conversations or next steps at your earliest convenience.

Thank you again for your time and consideration. I look forward to hearing from you soon.

Sincerely,
[Your Name]

Research Insights:
- Follow-Up Email

In [ ]:

# Set your auth token before starting the tunnel
# Retrieve the actual auth token from the environment variable or userdata
ngrok_auth_token = os.environ.get('NGROK_AUTH_TOKEN') or userdata.get('NGROK_AUTH_TOKEN')
ngrok.set_auth_token(ngrok_auth_token)  # Use the retrieved token

# Start ngrok tunnel
public_url = ngrok.connect(8000)
print(f"Public URL: {public_url}")

# Now run your FastAPI app
if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8000)

INFO:     Started server process [1795]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


Public URL: NgrokTunnel: "https://2a7b-34-16-159-183.ngrok-free.app" -> "http://localhost:8000"


## Project Evaluation

### Competencies Demonstrated
- ✅ REACT Architecture Understanding
- ✅ Prompt Engineering
- ✅ Tool Calling with LLMs
- ✅ Conversation Management

### Potential Improvements
- Enhanced context understanding
- More sophisticated draft generation
- Advanced research integration

In [ ]:
# Custom Email Draft Generation
def generate_custom_email_draft(context):
    """Allow users to generate custom email drafts"""
    result = assistant.analyze_email_context(context)

    print("🚀 Custom Email Draft Generated:")
    print(result['draft_content'])

    print("\n🔍 Research Insights:")
    for insight in result['research_insights']:
        print(f"- {insight['title']}: {insight['url']}")

# Example custom usage
custom_context = input("Enter your email context: ")
generate_custom_email_draft(custom_context)